In [18]:
#%%writefile snowflake_process.py
#!/usr/bin/env python
import import_ipynb
import snowflake.connector
import ast
from children_to_parent import child_to_parent


def snowflake_connection(user,
                         password,
                         account,
                         warehouse):
    ctx = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    )
    return ctx

def create_table(table_name = "DEV.RBAREIRO.HOUSEHOLD_VIEWERSHIP_CLASSIFICATION_EXP_LUID_PATH_2"):
    """Create table to save PARENT_NODE_ID and CHILD_NODE_ID
    Keyword arguments:
    table_name -- Table name
    """
    connector = snowflake_connection()
    cursor = connector.cursor()
    cursor.execute(f"drop table if exists {table_name}")

    sql = f"""
    CREATE  TABLE {table_name}
    (
    PARENT_NODE_ID INT ,
    CHILD_NODE_ID INT 
    )
    """
    cursor.execute(sql)
    cursor.close()
    connector.close()

def get_root_to_leaf(consumer_view_start_ds = '20220101', 
                     org_table_name ="DEV.JAYD.HOUSEHOLD_VIEWERSHIP_CLASSIFICATION_EXP_LUID_PATH",
                     limit = 200 ):
    """Select ROOT_TO_LEAF from a given table
    Keyword arguments:
    org_table_name -- Table to get ROOT_TO_LEAF
    limit -- SQL row limit
    """
    connector = snowflake_connection()
    cursor = connector.cursor()

    sql = f"""
        SELECT ROOT_TO_LEAF, EXP_LUID FROM {org_table_name} 
        where CONSUMER_VIEW_START_DS = {consumer_view_start_ds}
        limit {limit}
        """
    cursor.execute(sql)
    query_id = cursor.sfqid
    # Get the results from a query.
    cursor.get_results_from_sfqid(query_id)
    dataframe = cursor.fetchall()
    cursor.close()
    connector.close()
    return dataframe

def insert_values_into_table(values, 
                             table_name="DEV.RBAREIRO.HOUSEHOLD_VIEWERSHIP_CLASSIFICATION_EXP_LUID_PATH_2"):
    """Insert PARENT_NODE_ID and CHILD_NODE_ID
    Keyword arguments:
    values -- tuple data 
    table_name -- Table name
    """
    if values is None:
        print("No values to insert")
        return
    
    values = (''.join(str(values))).strip("]").strip("[")
    
    connector = snowflake_connection()
    cursor = connector.cursor()

    sql = f"""
        insert into {table_name}
        (CHILD_NODE_ID, PARENT_NODE_ID)
        values {values}
        """
    print("sql:", sql)
    cursor.execute(sql)
    print("Success!")
    cursor.close()
    connector.close()
    


if __name__ == "__main__":
    create_table()
    dataframe = get_root_to_leaf()

    parent_child = []
    #node = {}

    for elem in dataframe:
        parent_child.append( ast.literal_eval(elem[0])) # PARENT_NODE_ID, CHILD_NODE_ID
        #node[elem[1]]= ast.literal_eval(elem[0]) #EXP_LUID
    tree = child_to_parent(parent_child)
    insert_values_into_table(tree)

sql: 
        insert into DEV.RBAREIRO.HOUSEHOLD_VIEWERSHIP_CLASSIFICATION_EXP_LUID_PATH_2
        (CHILD_NODE_ID, PARENT_NODE_ID)
        values (1002, 1), (10022, 1002), (100220002, 10022), (100220005, 100220002), (100220011, 100220005), (100220023, 100220011), (100220047, 100220023), (100220095, 100220047), (100220190, 100220095), (100220380, 100220190), (100220761, 100220380), (1004, 1), (10041, 1004), (100410002, 10041), (100410004, 100410002), (100410008, 100410004), (100410016, 100410008), (100410032, 100410016), (100410064, 100410032), (100410128, 100410064), (100410257, 100410128), (100410514, 100410257), (100411028, 100410514), (100412056, 100411028), (100414112, 100412056), (1001, 1), (10010, 1001), (100100003, 10010), (100100007, 100100003), (100100015, 100100007), (100100031, 100100015), (100100063, 100100031), (100100126, 100100063), (100100252, 100100126), (100100504, 100100252), (100101009, 100100504), (100102018, 100101009), (100104036, 100102018), (1024, 1), (10240, 1

See the results in DEV.RBAREIRO.HOUSEHOLD_VIEWERSHIP_CLASSIFICATION_EXP_LUID_PATH_2 table (default table)